In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.ticker as ticker
from tqdm.notebook import tqdm

# Configure font sizes.
TICK_SIZE = 6
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 11
plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=TICK_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=TICK_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

MARKERSIZE = 3
LINEWIDTH = 0.5
plt.rc("lines", markersize=MARKERSIZE, linewidth=LINEWIDTH)
plt.rc("grid", linewidth=0.5)
plt.rc("grid", alpha=0.5)

# colorblind friendly colors: https://gist.github.com/thriveth/8560036
colors = ["#377eb8", "#ff7f00", "#4daf4a", "#f781bf", "#a65628", "#984ea3", "#999999", "#e41a1c", "#dede00"]

# how many questions does each dataset have?
N_QUESTIONS = {"bbh+gpqa+ifeval+math+musr" : 9574, "mmlu-pro" : 12032}

In [ ]:
# how many seeds did we have?
N_SEEDS = 100

# load in the relevant logs
logs_wo = pd.read_csv("logs/final/without_replacement_ablation/faq_without-replacement=True.csv")
logs_wr = pd.read_csv("logs/final/without_replacement_ablation/faq_without-replacement=False.csv")

# get RMSE calculations
logs_wo["rmse"] = np.sqrt(logs_wo.sq_err)
logs_wr["rmse"] = np.sqrt(logs_wr.sq_err)

# summarize the logs - get coverage + MSE 
summary_wo = logs_wo.groupby(["dataset", "prop_budget"]).mean().reset_index()
summary_wr = logs_wr.groupby(["dataset", "prop_budget"]).mean().reset_index()
summary_wo["coverage_serr"] = logs_wo.groupby(
    ["dataset", "prop_budget"]).std().reset_index().coverage / np.sqrt(N_SEEDS)
summary_wr["coverage_serr"] = logs_wr.groupby(
    ["dataset", "prop_budget"]).std().reset_index().coverage / np.sqrt(N_SEEDS)
summary_wo["rmse_serr"] = logs_wo.groupby(
    ["dataset", "prop_budget"]).std().reset_index().rmse / np.sqrt(N_SEEDS)
summary_wr["rmse_serr"] = logs_wr.groupby(
    ["dataset", "prop_budget"]).std().reset_index().rmse / np.sqrt(N_SEEDS)

In [ ]:
# we can put coverages in the appendix: let's just do a 2x2 of dataset x missingness-setting
fig, ax = plt.subplots(1, 2, dpi=400, figsize=(6.5, 2.0), sharex=True)

# let's do coverages first to show miscoverage
ax[0].errorbar(
    x = summary_wr.prop_budget * N_QUESTIONS["mmlu-pro"],
    y = summary_wr.coverage, yerr = summary_wr.coverage_serr, marker="o", 
    capsize=MARKERSIZE, capthick=1.0, color=colors[0])
ax[0].errorbar(
    x = summary_wo.prop_budget * N_QUESTIONS["mmlu-pro"],
    y = summary_wo.coverage, yerr = summary_wo.coverage_serr, marker="x", 
    capsize=MARKERSIZE, capthick=1.0, color=colors[1])
ax[0].grid(); ax[0].set_xlabel("Budget"); ax[0].set_title("Coverage")

# next let's show how RMSE really doesn't change much at a noticeable scale
ax[1].errorbar(
    x = summary_wr.prop_budget * N_QUESTIONS["mmlu-pro"],
    y = summary_wr.rmse, yerr = summary_wr.rmse_serr, marker="o", 
    capsize=MARKERSIZE, capthick=1.0, color=colors[0])
ax[1].errorbar(
    x = summary_wo.prop_budget * N_QUESTIONS["mmlu-pro"],
    y = summary_wo.rmse, yerr = summary_wo.rmse_serr, marker="x", 
    capsize=MARKERSIZE, capthick=1.0, color=colors[1])
ax[1].grid(); ax[1].set_xlabel("Budget"); ax[1].set_title("RMSE")
ax[1].set_yticks([0.004, 0.008, 0.012, 0.016, 0.020])
ax[1].set_yticklabels([0.004, 0.008, 0.012, 0.016, 0.020])

# custom legend
handles = [
    Line2D([], [], marker="o", color=colors[0], label="FAQ (Sampling With Replacement)"),
    Line2D([], [], marker="x", color=colors[1], label="FAQ (Sampling Without Replacement)"),
]
fig.legend(handles=handles, ncol=4, loc="lower center", bbox_to_anchor=(0.5, -0.08))

# beautify
plt.tight_layout()
plt.savefig("appendix_figures/6_with-vs-without-replacement.pdf", facecolor="white", bbox_inches="tight")
plt.show()